In [1]:
'''

https://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/issues/594

Homologação: http://acs-assist-sem-rag.nia.hm.bb.com.br/nia_semantica_acs/docs

Desenvolvimento: http://acs-assist-sem-rag.nia.desenv.bb.com.br/nia_semantica_acs/docs

nesse primeiro exemplo vamos trabalhar com agentes sem rag para maior liberdade e entendimento de como funciona agentes

documentação:

https://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/wikis/home


Base de conhecimento RAG

https://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?sw=auth#declara%c3%a7%c3%a3o-de-assistente
https://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?#indexa%C3%A7%C3%A3o-assistente

'''

'\n\nhttps://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/issues/594\n\nHomologação: http://acs-assist-sem-rag.nia.hm.bb.com.br/nia_semantica_acs/docs\n\nDesenvolvimento: http://acs-assist-sem-rag.nia.desenv.bb.com.br/nia_semantica_acs/docs\n\nnesse primeiro exemplo vamos trabalhar com agentes sem rag para maior liberdade e entendimento de como funciona agentes\n\ndocumentação:\n\nhttps://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/wikis/home\n\n\nBase de conhecimento RAG\n\nhttps://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?sw=auth#declara%c3%a7%c3%a3o-de-assistente\nhttps://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?#indexa%C3%A7%C3%A3o-assistente\n\n'

RAG: Retrieval-Augmented Generation (Geração Aumentada por Recuperação). Essa técnica combina a geração de texto com a recuperação de informações de fontes externas para melhorar a precisão e a relevância das respostas geradas pelos modelos de linguagem.

Exemplo:

Recuperação de Informações: Quando o modelo recebe uma consulta, ele primeiro busca informações relevantes em uma base de dados externa ou em documentos específicos.
Geração de Texto: Em seguida, o modelo usa essas informações recuperadas para gerar uma resposta mais precisa e informada.

Como nesse agente cadastramos sem RAG cabe ao deve passar as informações relevantes

exemplo pratico no BB:

Aqui está um relatório de auditoria especial. Quero que você identifique e descreva o modus operandi utilizado nas irregularidades encontradas.

**Conceito de Modus Operandi**:
Modus operandi é um termo usado para descrever os métodos e processos utilizados por uma pessoa ou grupo para cometer uma fraude ou irregularidade.

**Exemplos de Modus Operandi**:
1. Uso de documentos falsificados para abrir contas bancárias.
2. Manipulação de registros financeiros para ocultar desfalques.
3. Uso de identidades falsas para realizar transações fraudulentas.

**Relatório de Auditoria Especial**:
[Insira o texto do relatório aqui]

Com base nas informações acima, identifique e descreva o modus operandi utilizado nas irregularidades encontradas no relatório.


In [2]:
# controlar o máximo de envio de informação
import tiktoken

In [3]:
import pandas as pd
import os
import sys
import numpy 
import json
import time
import requests
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, types
from IPython.display import display, JSON

In [4]:
# Carregue o encoding para o modelo GPT-3.5-turbo (fica proximo do modelo atual)
encoding = tiktoken.encoding_for_model("gpt-4")

In [5]:
################  Amostra manual  ##########

Nesse exemplo pode ser usado para exploração e entendimento do negócio, entender como gerar um prompt e ranalisar se o resultado de retorno atende o objetivo:

O Banco disponibiliza uma área específica para isso no caminho: plataforma -> tecnologia -> Engenharia de Prompts -> escolher seu prompt -> testar prompt.

In [6]:
 orientacao_prompt = '''Você é um assistente de auditoria especializado em vincular processos corporativos de auditoria. Sua tarefa é classificar processos de apuração de irregularidades em categorias específicas de risco corporativo e retornar o resultado no formato JSON. As categorias de risco são:
    1.Risco de Estratégia; 2.Risco de Crédito; 3.Risco Atuarial; 4.Risco de IRRBB; 5.Risco de Mercado; 6.Risco de Liquidez; 7.Risco de Contágio; 18.Risco Social, Ambiental e Climático; 45.Risco Cibernético; 48.Risco de Conformidade; 49.Risco de Terceiros; 50.Risco Legal; 776.Risco de Conduta; 777.Risco de Modelo; 778.Risco de Segurança; 779.Risco de TI
    Classifique a apuração de irregularidades em uma ou mais das categorias de risco corporativo listadas acima. Para cada categoria, indique "Sim" ou "Não". Além disso, crie uma categoria chamada regra_classificacao que descreve qual a lógica que você usou para classificar.
    Importante:
    1) A base de conhecimento possui "Definição Principal" e "Definição Secundária", delimitadas entre [] sendo que a classificação de risco precisa observar obrigatóriamente a "Definição Principal" sendo que a "Definição Secundária" está contida na "Definição Principal". 
    Exemplo: [Definição Principal:Possibilidade de perdas decorrentes do relacionamento do Banco com terceiros.]
    2) A classificação de riscos a ser realizada precisa dar maior peso à classificação mencionada no item 1).
    '''

In [7]:
formato_saida = '''
{
    "regra_classificacao": "foi classificada como sim Risco Operacional, Risco Financeiro porque houve uma falha no sistema de processamento, levando a um prejuízo financeiro significativo.",
    "risco_estrategia": "Não",
    "risco_credito": "Não",
    "risco_atuarial": "Sim",
    "risco_IRRBB": "Não",
    "risco_mercado": "Não",
    "risco_liquidez": "Não",
    "risco_contagio": "Não",
    "risco_social_ambiental_climatico": "Não",
    "risco_cibernetico": "Não",
    "risco_conformidade": "Não",
    "risco_terceiros": "Não",
    "risco_legal": "Não",
    "risco_conduta": "Não",
    "risco_modelo": "Não",
    "risco_seguranca": "Não",
    "risco_ti": "Não"
} 

OBS.: Envie o JSON em formato puro sem vestígios de markdown, remove a marcação abaixo:
[REMOVER]:
```json
```
'''

In [8]:
irregularidades = '''1 Vinculação de comprovantes espúrios de endereço no cadastro de clientes 2 Registro de produção agropecuária baseadas em cessões fictícias ou inexistentes de áreas rurais no cadastro de clientes 3 Abertura de contas correntes e ou cálculo de limite de crédito com base em informação cadastral fictícia 1 Atuação gerencial frente à agência 3417 Vieira de Carvalho SP favorecendo a abertura de contas e o estabelecimento de limite de crédito para empresas recém constituídas sem análise e avaliação crítica das informações cadastrais apresentadas visando a liberação de crédito por meio de operações que não apresentavam perspectivas de retorno dos capitais 2 Contratação e liberação de recursos de operações de Crédito Pessoa Jurídica com irregularidades caracterizadas pela insuficiência de garantias e por Instrumentos de Crédito ausentes ou com assinaturas insuficientes e ou incompatíveis 3 Beneficiamento de funcionário BB com recursos originados em operações de Crédito Pessoa Jurídica contratadas e liberadas na agência com sua participação 4 Abertura de conta corrente Pessoa Física com documentos falsos e cadastramento de dispositivos móveis sem autorização formal da titular viabilizando a contratação de operações de CDC Consignado e o direcionamento de parte dos recursos liberados em benefício de funcionário BB 5 Movimentação de contas correntes Pessoa Física sem anuência dos titulares com consequente apropriação indébita de valores Abertura de contas PJ com liberação imediata de crédito reescalonamento com carência de 6 meses e sequencial inadimplemento sendo que para duas das contas abertas houve questionamento formal dos clientes que não reconhecem a abertura das contas com contratação de produtos Inclusão de documentos espúrios na Plataforma BB Dossiê Eletrônico DEL DEOC e Cartão de Autógrafo Eletrônico AEL cujas assinaturas dos responsáveis legais titulares apontam evidências de falsidade conforme demonstrado nos Laudos de Exame Grafotécnico Registros de abertura de contas correntes PJ e cadastramentos de senhas de 6 e 8 dígitos inobservando o princípio Conheça seu Cliente e contrariando o Código de Ética as Normas de Conduta e procedimentos operacionais do Banco A envolvida não foi afastada uma vez que não apresentava risco para a apuração ou para o bom funcionamento da dependência a Registros irregulares de abertura de contas para clientes pessoa física que posteriormente foram movimentadas por funcionária b Cadastramento de senhas de 06 e 08 dígitos de contas correntes pessoas físicas via Plataforma de Negócios e cadastramento de códigos de acesso via TAA sem a presença dos clientes que posteriormente foram utilizadas para a formalização de contratos contratação de operações de crédito e movimentação das contas c Realização de registro de cadastramento ou substituição de perfil biométrico sem a presença dos clientes d Cadastramento de equipamentos via sistema ATO para acesso a contas de clientes pessoa física sem autorização formal e Cadastramento de endereços residenciais para clientes com base em documentos cujas assinaturas divergem dos padrões ou com outras irregularidades f Cadastramento de ocupação e renda para clientes com base em documentos cujas assinaturas divergem dos padrões ou com outras irregularidades g Acolhimento Vinculação de Cartões de Autógrafos para clientes Pessoa Física cujas assinaturas divergem dos padrões h Vinculação de cartão de crédito pré impresso ou realização de entrega de cartões sem autorização dos clientes i Contratação de operações de crédito para clientes PF sem autorização formal cujos valores beneficiaram a funcionária envolvida j Realização de saques transferências eletrônicas de valores transferências via PIX TED pagamento de boletos títulos compras com cartão de débito via TAA Banco 24 horas ou Mobile à débito de contas correntes de clientes irregularmente abertas e movimentadas pela funcionária sem poderes para tal k Realização de compras pagamento de boletos títulos e saques com cartão de crédito de clientes sem poderes para tal l Encerramento de conta corrente e liquidação de operações sem solicitação formal do cliente OBS a principal envolvida foi afastada dos serviços no dia 19 05 2023	
'''

abaixo, segue exemplo de rag (base de conhecimento), se ela for estática crie agente com base de conhecimento, se for dinamica ou nao tiver crie sem rag

In [9]:
exemplo_rag_dinamica = '''
"[Entidade Ligada] ATIVOS S. A. - SECURITIZADORA DE CRÉDITOS / GESTÃO DE NEGÓCIOS"
"[Entidade Ligada] ATIVOS S. A. - SECURITIZADORA DE CRÉDITOS / GESTÃO EMPRESARIAL"
"[Entidade Ligada] ATIVOS S. A. - SECURITIZADORA DE CRÉDITOS / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] ATIVOS S. A. - SECURITIZADORA DE CRÉDITOS / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] ATIVOS S.A. - GESTÃO DE COBRANÇA E RECUPERAÇÃO DE CRÉDITO / GESTÃO DA SEGURANÇA, RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] ATIVOS S.A. - GESTÃO DE COBRANÇA E RECUPERAÇÃO DE CRÉDITO / SISTEMA DE GESTÃO"
"[Entidade Ligada] BANCO PATAGONIA S.A. / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BANCO PATAGONIA S.A. / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / CONTABILIDADE"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / GESTÃO DE NEGÓCIOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / GESTÃO DE RISCOS E CONTROLES"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / GESTÃO EMPRESARIAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / LOGÍSTICA E SUPORTE OPERACIONAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / PREVENÇÃO E COMBATE À LAVAGEM DE DINHEIRO E AO FINANCIAMENTO AO TERRORISMO"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / SEGURANÇA EMPRESARIAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / SERVIÇOS JURÍDICOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / TECNOLOGIA DE INFORMAÇÃO"
"[Entidade Ligada] BB ADMINISTRADORA DE CARTÕES DE CRÉDITO S.A. / TESOURARIA E FINANÇAS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / CONTABILIDADE"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / GESTÃO DE PESSOAS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / GESTÃO EMPRESARIAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / LOGÍSTICA E SUPORTE OPERACIONAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / NEGÓCIOS DE CONSÓRCIOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / OPERACIONALIZAÇÃO DE CONSÓRCIOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / PREVENÇÃO E COMBATE À LAVAGEM DE DINHEIRO"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / SEGURANÇA EMPRESARIAL"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / SERVIÇOS JURÍDICOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / TECNOLOGIA DA INFORMAÇÃO"
"[Entidade Ligada] BB ADMINISTRADORA DE CONSÓRCIOS S.A. / TESOURARIA E FINANÇAS"
"[Entidade Ligada] BB ASSET / ADMINISTRAÇÃO FIDUCIÁRIA DE FUNDOS DE INVESTIMENTO"
"[Entidade Ligada] BB ASSET / ANÁLISE DA PERFORMANCE E REVISÃO DO GERENCIAMENTO DE RISCOS"
"[Entidade Ligada] BB ASSET / COMERCIALIZAÇÃO E DESENVOLVIMENTO DE PRODUTOS"
"[Entidade Ligada] BB ASSET / ESTRATÉGIA E OBJETIVOS"
"[Entidade Ligada] BB ASSET / GESTÃO CORPORATIVA"
"[Entidade Ligada] BB ASSET / GESTÃO DA INFORMAÇÃO, COMUNICAÇÃO E DIVULGAÇÃO"
"[Entidade Ligada] BB ASSET / GESTÃO DA PERFORMANCE"
"[Entidade Ligada] BB ASSET / GESTÃO DE ATIVOS DE FUNDOS DE INVESTIMENTO"
"[Entidade Ligada] BB ASSET / GESTÃO DE RISCOS CORPORATIVOS, CONTROLES INTERNOS E CONFORMIDADE"
"[Entidade Ligada] BB ASSET / GESTÃO DE RISCOS DE FUNDOS DE INVESTIMENTO, CONTROLES INTERNOS E CONFORMIDADE"
"[Entidade Ligada] BB ASSET / GESTÃO ECONÔMICO-FINANCEIRA"
"[Entidade Ligada] BB ASSET / GESTÃO EMPRESARIAL"
"[Entidade Ligada] BB ASSET / GESTÃO TECNOLOGIA E INFORMAÇÃO"
"[Entidade Ligada] BB ASSET / GESTÃO, ADMINISTRAÇÃO E COMERCIALIZAÇÃO DE FUNDOS E CARTEIRAS"
"[Entidade Ligada] BB ASSET / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB ASSET / GOVERNANÇA E CULTURA"
"[Entidade Ligada] BB ASSET / SISTEMA DE GOVERNANÇA, GESTÃO, GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] BB BANCO DE INVESTIMENTO S.A. / GESTÃO DE RECURSOS E DE NEGÓCIOS"
"[Entidade Ligada] BB BANCO DE INVESTIMENTO S.A. / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / CONTABILIDADE"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / GESTÃO DE RISCOS"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / GESTÃO EMPRESARIAL"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / LOGÍSTICA E SUPORTE OPERACIONAL"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / PREVENÇÃO E COMBATE À LAVAGEM DE DINHEIRO E AO FINANCIAMENTO AO TERRORISMO"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / SEGURANÇA EMPRESARIAL"
"[Entidade Ligada] BB ELO CARTÕES PARTICIPAÇÕES S.A. / TESOURARIA E FINANÇAS"
"[Entidade Ligada] BB LEASING SA - ARRENDAMENTO MERCANTIL / GESTÃO DE NEGÓCIOS"
"[Entidade Ligada] BB LEASING SA - ARRENDAMENTO MERCANTIL / GESTÃO EMPRESARIAL"
"[Entidade Ligada] BB LEASING SA - ARRENDAMENTO MERCANTIL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB LEASING SA - ARRENDAMENTO MERCANTIL / SISTEMAS DE CONTROLES INTERNOS"
"[Entidade Ligada] BB PREVIDÊNCIA / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BB PREVIDÊNCIA / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] BB SECURITIES LIMITED / ACCOUNTING"
"[Entidade Ligada] BB SECURITIES LIMITED / ADMINISTRATIVE ORGANIZATION"
"[Entidade Ligada] BB SECURITIES LIMITED / ADMINISTRAÇÃO DO PATRIMÔNIO"
"[Entidade Ligada] BB SECURITIES LIMITED / AJUSTES E APURAÇÃO DO RESULTADO CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LIMITED / ANTI-MONEY LAUNDERING-AML"
"[Entidade Ligada] BB SECURITIES LIMITED / ARQUITETURA ORGANIZACIONAL"
"[Entidade Ligada] BB SECURITIES LIMITED / BACK OFFICE"
"[Entidade Ligada] BB SECURITIES LIMITED / CAPACITAÇÃO DE FUNCIONÁRIOS"
"[Entidade Ligada] BB SECURITIES LIMITED / CAPITAL MARKETS"
"[Entidade Ligada] BB SECURITIES LIMITED / COMPLIANCE"
"[Entidade Ligada] BB SECURITIES LIMITED / COMPRAS E ADMINISTRAÇÃO DE CONTRATOS"
"[Entidade Ligada] BB SECURITIES LIMITED / COMUNICAÇÃO DE INDÍCIOS DE LAVAGEM DE DINHEIRO"
"[Entidade Ligada] BB SECURITIES LIMITED / COMUNICAÇÃO INTERNA"
"[Entidade Ligada] BB SECURITIES LIMITED / CORPORATE GOVERNANCE"
"[Entidade Ligada] BB SECURITIES LIMITED / CORPORATE SECURITY"
"[Entidade Ligada] BB SECURITIES LIMITED / ESCRITURAÇÃO CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LIMITED / ESTRATÉGIA E PLANEJAMENTO"
"[Entidade Ligada] BB SECURITIES LIMITED / ESTRUTURAÇÃO DO SISTEMA CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LIMITED / EVIDENCIAÇÃO DOS DEMONSTRATIVOS CONTÁBEIS"
"[Entidade Ligada] BB SECURITIES LIMITED / EXECUÇÃO DA GESTÃO DA LIQUIDEZ"
"[Entidade Ligada] BB SECURITIES LIMITED / FINANCE"
"[Entidade Ligada] BB SECURITIES LIMITED / GERENCIAMENTO"
"[Entidade Ligada] BB SECURITIES LIMITED / GERENCIAMENTO DA CONTINUIDADE DOS SERVIÇOS"
"[Entidade Ligada] BB SECURITIES LIMITED / GERENCIAMENTO DE RECURSOS TERCEIRIZADOS"
"[Entidade Ligada] BB SECURITIES LIMITED / GERENCIAMENTO DO RISCO DE CRÉDITO"
"[Entidade Ligada] BB SECURITIES LIMITED / GERENCIAMENTO DO RISCO DE MERCADO E LIQUIDEZ"
"[Entidade Ligada] BB SECURITIES LIMITED / GESTÃO DE ATENDIMENTO A CLIENTES DE TI"
"[Entidade Ligada] BB SECURITIES LIMITED / GESTÃO DE NÍVEIS DE SERVIÇO"
"[Entidade Ligada] BB SECURITIES LIMITED / GESTÃO DO AMBIENTE FÍSICO"
"[Entidade Ligada] BB SECURITIES LIMITED / GESTÃO FISCO TRIBUTÁRIA"
"[Entidade Ligada] BB SECURITIES LIMITED / GLOBAL RISK MANAGEMENT"
"[Entidade Ligada] BB SECURITIES LIMITED / HUMAN RESOURCES"
"[Entidade Ligada] BB SECURITIES LIMITED / IDENTIFICAÇÃO DE CLIENTES"
"[Entidade Ligada] BB SECURITIES LIMITED / IDENTIFICAÇÃO E DESENVOLVIMENTO PROFISSIONAL"
"[Entidade Ligada] BB SECURITIES LIMITED / INFORMATION TECHNOLOGY"
"[Entidade Ligada] BB SECURITIES LIMITED / LOGISTICS"
"[Entidade Ligada] BB SECURITIES LIMITED / OPERACIONALIZAÇÃO E DISTRIBUIÇÃO"
"[Entidade Ligada] BB SECURITIES LIMITED / PARTICIPAÇÕES ACIONÁRIAS E SOCIETÁRIAS"
"[Entidade Ligada] BB SECURITIES LIMITED / PLANO DE PENSÃO COMPLEMENTAR"
"[Entidade Ligada] BB SECURITIES LIMITED / POLÍTICAS E DIRETRIZES"
"[Entidade Ligada] BB SECURITIES LIMITED / PRECIFICAÇÃO"
"[Entidade Ligada] BB SECURITIES LIMITED / PRINCÍPIO CONHEÇA SEU FUNCIONÁRIO"
"[Entidade Ligada] BB SECURITIES LIMITED / PRIORIZAÇÃO DE DEMANDAS POR SISTEMAS INFORMATIZADOS"
"[Entidade Ligada] BB SECURITIES LIMITED / REALIZAÇÃO DE OPERAÇÕES"
"[Entidade Ligada] BB SECURITIES LIMITED / REGISTRO DAS TRANSAÇÕES"
"[Entidade Ligada] BB SECURITIES LIMITED / RELAÇÕES TRABALHISTAS"
"[Entidade Ligada] BB SECURITIES LIMITED / REPRESENTAÇÕES MANDATÁRIAS"
"[Entidade Ligada] BB SECURITIES LIMITED / REQUISITOS LEGAIS E REGULAMENTARES PARA FUNCIONAMENTO"
"[Entidade Ligada] BB SECURITIES LIMITED / RETRIBUIÇÃO E DINÂMICA LABORAL"
"[Entidade Ligada] BB SECURITIES LIMITED / RISCO OPERACIONAL"
"[Entidade Ligada] BB SECURITIES LIMITED / SEGURANÇA DA INFORMAÇÃO"
"[Entidade Ligada] BB SECURITIES LIMITED / SEGURANÇA FÍSICA E PATRIMONIAL"
"[Entidade Ligada] BB SECURITIES LIMITED / SISTEMA NORMATIVO"
"[Entidade Ligada] BB SECURITIES LIMITED / VALIDAÇÃO DOS SALDOS CONTÁBEIS"
"[Entidade Ligada] BB SECURITIES LIMITED / ÉTICA E CONDUTA"
"[Entidade Ligada] BB SECURITIES LLC / ACCOUNTING"
"[Entidade Ligada] BB SECURITIES LLC / AJUSTES E OBTENÇÃO DO RESULTADO CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LLC / APLICATIVO UTILIZADO NA CONTABILIDADE"
"[Entidade Ligada] BB SECURITIES LLC / BSA/AML AND OFAC"
"[Entidade Ligada] BB SECURITIES LLC / CAPACITAÇÃO DE FUNCIONÁRIOS"
"[Entidade Ligada] BB SECURITIES LLC / CAPITAL MARKETS"
"[Entidade Ligada] BB SECURITIES LLC / COMUNICAÇÃO DE INDÍCIOS DE LAVAGEM DE DINHEIRO"
"[Entidade Ligada] BB SECURITIES LLC / CORPORATE COMPLIANCE"
"[Entidade Ligada] BB SECURITIES LLC / CORPORATE GOVERNANCE"
"[Entidade Ligada] BB SECURITIES LLC / ELABORAÇÃO E PUBLICAÇÃO DOS RELATÓRIOS CONTÁBEIS"
"[Entidade Ligada] BB SECURITIES LLC / FINANCE"
"[Entidade Ligada] BB SECURITIES LLC / HUMAN RESOURCES"
"[Entidade Ligada] BB SECURITIES LLC / IDENTIFICAÇÃO DE CLIENTES"
"[Entidade Ligada] BB SECURITIES LLC / INFORMATION TECHNOLOGY"
"[Entidade Ligada] BB SECURITIES LLC / LOGISTICS"
"[Entidade Ligada] BB SECURITIES LLC / MONITORAMENTO DAS TRANSAÇÕES REALIZADAS"
"[Entidade Ligada] BB SECURITIES LLC / ORGANIZAÇÃO DO SISTEMA CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LLC / POLÍTICAS E DIRETRIZES"
"[Entidade Ligada] BB SECURITIES LLC / PRINCÍPIO CONHEÇA SEU FUNCIONÁRIO"
"[Entidade Ligada] BB SECURITIES LLC / REGISTRO DAS TRANSAÇÕES"
"[Entidade Ligada] BB SECURITIES LLC / REGISTRO DAS TRANSAÇÕES NO SISTEMA CONTÁBIL"
"[Entidade Ligada] BB SECURITIES LLC / RETENÇÃO, REPASSE E PAGAMENTOS DE IMPOSTOS"
"[Entidade Ligada] BB SECURITIES LLC / RISK MANAGEMENT"
"[Entidade Ligada] BB SECURITIES LLC / SECURITY"
"[Entidade Ligada] BB SECURITIES LLC / VALIDAÇÃO DOS SALDOS CONTÁBEIS"
"[Entidade Ligada] BB SEGURIDADE / SISTEMA DE GOVERNANÇA, GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] BB TECNOLOGIA E SERVIÇOS / SISTEMA DE GOVERNANÇA, GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] BBCI HOLDING / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] BBCI HOLDING / SUPORTE OPERACIONAL"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / CONTROLADORIA"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / ESTRATÉGIA CORPORATIVA"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / FINANÇAS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / FINANÇAS, CONTABILIDADE E CONTROLADORIA"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GESTÃO DE NEGÓCIOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GESTÃO DE PESSOAS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GESTÃO DE PESSOAS E SERVIÇOS JURÍDICOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GOVERNANÇA E GESTÃO ESTRATÉGICA"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GOVERNANÇA E RISCOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / GOVERNANÇA EMPRESARIAL"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / LIQUIDAÇÃO EXTRAJUDICIAL"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / LOGÍSTICA E SERVIÇOS JURÍDICOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / PROCESSOS INTERNOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] BBTUR VIAGENS E TURISMO LTDA. / TECNOLOGIA"
"[Entidade Ligada] CASSI - CAIXA DE ASSISTÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] CASSI - CAIXA DE ASSISTÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / CONTROLES INTERNOS"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GESTÃO ADMINISTRATIVA E FINANCEIRA"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GESTÃO ATUARIAL, CUSTEIO E CADASTRO"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GESTÃO DE BENEFÍCIOS E CONTRIBUIÇÕES"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GESTÃO DE INVESTIMENTOS"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] ECONOMUS INSTITUTO DE SEGURIDADE SOCIAL / GOVERNANÇA E CONTROLES INTERNOS"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / FINANÇAS E CONTROLADORIA"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / GESTÃO ADMINISTRATIVA"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / GESTÃO DE PROJETOS SOCIAIS"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / GOVERNANÇA CORPORATIVA E GESTÃO EMPRESARIAL"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / SERVIÇOS JURÍDICOS"
"[Entidade Ligada] FBB - FUNDAÇÃO BANCO DO BRASIL / SISTEMA DE GOVERNANÇA, GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO ADMINISTRATIVA"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO ATUARIAL, CUSTEIO E CADASTRO"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO DE BENEFÍCIOS E CONTRIBUIÇÕES"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO DE INVESTIMENTOS"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GESTÃO FINANCEIRA"
"[Entidade Ligada] FUSESC - FUNDAÇÃO CODESC DE SEGURIDADE SOCIAL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO ADMINISTRATIVA"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO ATUARIAL, CUSTEIO E CADASTRO"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO DE BENEFÍCIOS E CONTRIBUIÇÕES"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO DE INVESTIMENTOS"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GESTÃO FINANCEIRA"
"[Entidade Ligada] PREVBEP - BEP CAIXA DE PREVIDÊNCIA SOCIAL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / ATUAÇÃO JUNTO ÀS EMPRESAS PARTICIPADAS E DEMAIS SÓCIOS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / AVALIAÇÃO DAS EMPRESAS PARTICIPADAS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / CONSELHEIROS DAS EMPRESAS PARTICIPADAS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GESTÃO ADMINISTRATIVA E FINANCEIRA"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GESTÃO ATUARIAL, CUSTEIO E CADASTRO"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GESTÃO DE BENEFÍCIOS E CONTRIBUIÇÕES"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GESTÃO DE INVESTIMENTOS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GOVERNANÇA CORPORATIVA"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GOVERNANÇA CORPORATIVA DE ENTIDADES LIGADAS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / GOVERNANÇA E CONTROLES INTERNOS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / POLÍTICAS E ESTRATÉGIAS"
"[Entidade Ligada] PREVI - CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL / SISTEMA DE CONTROLES INTERNOS"
"[Entidade Ligada] SIM PLANO DE SAÚDE - CAIXA DE ASSISTÊNCIA DOS EMPREGADOS DOS SISTEMAS BESC E CODESC, DO BADESC E DA FUSESC / GESTÃO ADMINISTRATIVA"
"[Entidade Ligada] SIM PLANO DE SAÚDE - CAIXA DE ASSISTÊNCIA DOS EMPREGADOS DOS SISTEMAS BESC E CODESC, DO BADESC E DA FUSESC / GESTÃO DE RISCOS E CONTROLES INTERNOS"
"[Entidade Ligada] SIM PLANO DE SAÚDE - CAIXA DE ASSISTÊNCIA DOS EMPREGADOS DOS SISTEMAS BESC E CODESC, DO BADESC E DA FUSESC / GESTÃO DOS PLANOS DE SAÚDE"
"[Entidade Ligada] SIM PLANO DE SAÚDE - CAIXA DE ASSISTÊNCIA DOS EMPREGADOS DOS SISTEMAS BESC E CODESC, DO BADESC E DA FUSESC / GOVERNANÇA CORPORATIVA"
"[Outros] BANCO DO BRASIL AG - VIENA / CORPORATE GOVERNANCE"
"[Outros] BANCO DO BRASIL AG - VIENA / INTERNAL CONTROL SYSTEM"
"[Outros] BB AMÉRICAS / ACCOUNTING"
"[Outros] BB AMÉRICAS / BSA/AML/OFAC"
"[Outros] BB AMÉRICAS / CARD SERVICES"
"[Outros] BB AMÉRICAS / CORPORATE COMPLIANCE"
"[Outros] BB AMÉRICAS / CORPORATE GOVERNANCE"
"[Outros] BB AMÉRICAS / CREDIT"
"[Outros] BB AMÉRICAS / DEPOSITS"
"[Outros] BB AMÉRICAS / DISTRIBUTION CHANNELS"
"[Outros] BB AMÉRICAS / HUMAN RESOURCE"
"[Outros] BB AMÉRICAS / INFORMATION TECHNOLOGY"
"[Outros] BB AMÉRICAS / LOGISTICS"
"[Outros] BB AMÉRICAS / RISK MANAGEMENT"
"[Outros] BB AMÉRICAS / SECURITY"
"[Outros] BB AMÉRICAS / SERVICES"
"[Outros] BB AMÉRICAS / TREASURY"
"[Outros] NEGÓCIOS NO EXTERIOR / BB JAPAN  (TOKYO, GOTANDA, HAMAMATSU, NAGOYA, 2ND CONTROL LINE)"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Assunção"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Comprehensiveness of risk assessment and status of senior management involvement (Including the effectiveness of committee functions)"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Frankfurt"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Grand Cayman"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Londres"
"[Outros] OPERACIONALIZAÇÃO DE NEGÓCIOS NO EXTERIOR / Operating status of IT system including the effectiveness of detection standards (Including validation of IT system effectiveness)"
"[Outros] OPERAÇÃO INTERNACIONAL / GESTÃO DE CLIENTES"
"[Processo corporativo] AGRONEGÓCIOS / COBRANÇA E RECUPERAÇÃO DO CRÉDITO DE AGRONEGÓCIO"
"[Processo corporativo] AGRONEGÓCIOS / ESTRATÉGIAS E SOLUÇÕES PARA O AGRONEGÓCIO"
"[Processo corporativo] AGRONEGÓCIOS / OPERACIONALIZAÇÃO DO CRÉDITO DE AGRONEGÓCIO"
"[Processo corporativo] AGRONEGÓCIOS / SUBVENÇÕES E EXIGIBILIDADES DO AGRONEGÓCIO"
"[Processo corporativo] CAPTAÇÃO / CAPTAÇÕES ESTRUTURADAS"
"[Processo corporativo] CAPTAÇÃO / GESTÃO DO FUNDING E EXIGIBILIDADES"
"[Processo corporativo] CAPTAÇÃO / PRODUTOS DE CAPTAÇÃO"
"[Processo corporativo] CONTABILIDADE SOCIETÁRIA / CONTROLE CONTÁBIL"
"[Processo corporativo] CONTABILIDADE SOCIETÁRIA / ESCRITURAÇÃO CONTÁBIL"
"[Processo corporativo] CONTABILIDADE SOCIETÁRIA / EVIDENCIAÇÃO CONTÁBIL"
"[Processo corporativo] CONTABILIDADE SOCIETÁRIA / GESTÃO DA IMPARIDADE E DE PROVISÕES"
"[Processo corporativo] CONTROLADORIA / CONTROLE ORÇAMENTÁRIO"
"[Processo corporativo] CONTROLADORIA / GESTÃO DE CUSTOS E DESPESAS"
"[Processo corporativo] CONTROLADORIA / GESTÃO DE RESULTADO E DESEMPENHO"
"[Processo corporativo] CONTROLADORIA / INFORMAÇÕES GERENCIAIS"
"[Processo corporativo] CRÉDITO DE VAREJO / COBRANÇA E RECUPERAÇÃO DO CRÉDITO DE VAREJO"
"[Processo corporativo] CRÉDITO DE VAREJO / ESTRATÉGIAS, PRODUTOS E SOLUÇÕES PARA O CRÉDITO DE VAREJO"
"[Processo corporativo] CRÉDITO DE VAREJO / OPERACIONALIZAÇÃO DO CRÉDITO DE VAREJO"
"[Processo corporativo] CULTURA E PESSOAS / DINÂMICAS DE TRABALHO, SAÚDE E SEGURANÇA"
"[Processo corporativo] CULTURA E PESSOAS / ESTRATÉGIA DE CULTURA E PESSOAS"
"[Processo corporativo] CULTURA E PESSOAS / RETRIBUIÇÃO E INCENTIVOS"
"[Processo corporativo] CULTURA E PESSOAS / TALENTOS E CARREIRA"
"[Processo corporativo] CULTURA E PESSOAS / ÉTICA E RELAÇÕES DE TRABALHO"
"[Processo corporativo] DISTRIBUIÇÃO DE PRODUTOS DE INVESTIMENTO PARA CLIENTES / ASSESSORIA EM INVESTIMENTOS A CLIENTES"
"[Processo corporativo] DISTRIBUIÇÃO DE PRODUTOS DE INVESTIMENTO PARA CLIENTES / GESTÃO DA DISTRIBUIÇÃO DE FUNDOS DE INVESTIMENTO"
"[Processo corporativo] DISTRIBUIÇÃO DE PRODUTOS DE INVESTIMENTO PARA CLIENTES / SERVIÇOS FIDUCIÁRIOS PARA FUNDOS DE INVESTIMENTO"
"[Processo corporativo] FINANÇAS CORPORATIVAS / AVALIAÇÃO DE NEGÓCIOS E INVESTIMENTOS"
"[Processo corporativo] FINANÇAS CORPORATIVAS / GESTÃO DA LIQUIDEZ"
"[Processo corporativo] FINANÇAS CORPORATIVAS / GESTÃO DE ATIVOS E PASSIVOS"
"[Processo corporativo] FINANÇAS CORPORATIVAS / GESTÃO DE PRODUTOS FINANCEIROS"
"[Processo corporativo] FINANÇAS CORPORATIVAS / PRECIFICAÇÃO DE PRODUTOS E SERVIÇOS"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / DETECTAR EVENTOS DE SEGURANÇA CIBERNÉTICA"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / IDENTIFICAR AMEAÇAS DE SEGURANÇA AO AMBIENTE CIBERNÉTICO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / IMPLEMENTAÇÃO DA SEGURANÇA DA INFORMAÇÃO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / MONITORAMENTO DA SEGURANÇA DA INFORMAÇÃO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / PLANEJAMENTO DA SEGURANÇA DA INFORMAÇÃO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / PROTEGER O AMBIENTE CIBERNÉTICO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / RECUPERAR RECURSOS OU SERVIÇOS IMPACTADOS POR INCIDENTES DE SEGURANÇA CIBERNÉTICA"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / RESPONDER INCIDENTES DE SEGURANÇA CIBERNÉTICA"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / REVISÃO DA SEGURANÇA DA INFORMAÇÃO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / SEGURANÇA CIBERNÉTICA"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / SEGURANÇA DA INFORMAÇÃO"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / SEGURANÇA DE AMBIENTES E PESSOAS"
"[Processo corporativo] GESTÃO DA SEGURANÇA CORPORATIVA / SEGURANÇA EM PRODUTOS E CANAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS AUTOMATIZADOS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS COMPARTILHADOS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS DE TERCEIROS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS PRESENCIAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS PRÓPRIOS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CANAIS REMOTOS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CONHECIMENTO DO CLIENTE"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CORRESPONDENTES"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / CRM E B-COMMERCE"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / ESTRATÉGIA DE CANAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / ESTRATÉGIA E MODELAGEM DE CANAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / ESTRATÉGIAS E MODELOS PARA TARIFAS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / GESTÃO DA FORÇA DE VENDAS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / GESTÃO DE CARTEIRAS VAREJO"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / GESTÃO DO ATENDIMENTO VAREJO"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / GESTÃO DO CADASTRO"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / GESTÃO DO PROGRAMA DE RELACIONAMENTO"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / MODELAGEM DE CANAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / MONITORAMENTO E CONTROLE DE TARIFAS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / MONITORAMENTO E CONTROLE EM CANAIS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / OPERACIONALIZAÇÃO DE TARIFAS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / OUVIDORIA EXTERNA"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / RELACIONAMENTO COM O CLIENTE VAREJO"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / SERVIÇO DE ATENDIMENTO AO CONSUMIDOR - SAC"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / SUSTENTABILIDADE DOS NEGÓCIOS E GESTÃO DE CONSEQUÊNCIAS"
"[Processo corporativo] GESTÃO DE CLIENTES E DE CANAIS / TARIFAS"
"[Processo corporativo] GESTÃO FISCO-TRIBUTÁRIA / BANCO COMO CONTRIBUINTE TRIBUTÁRIO"
"[Processo corporativo] GESTÃO FISCO-TRIBUTÁRIA / BANCO COMO RESPONSÁVEL TRIBUTÁRIO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / AVALIAÇÃO DE CAPITAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DE CAPITAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DE RISCOS"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO ATUARIAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO CIBERNÉTICO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE CONDUTA"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE CONFORMIDADE"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE CONTÁGIO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE CRÉDITO, CONTRAPARTE E CONCENTRAÇÃO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE ESTRATÉGIA"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE IRRBB"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE LIQUIDEZ"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE MERCADO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE MODELO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE REPUTAÇÃO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE SEGURANÇA"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE TERCEIROS"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO DE TI"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO LEGAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DO RISCO OPERACIONAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / GESTÃO DOS RISCOS SOCIAL, AMBIENTAL E CLIMÁTICO"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / MONITORAMENTO  DE CAPITAL"
"[Processo corporativo] GESTÃO INTEGRADA DE RISCOS E DE CAPITAL / TESTE DE ESTRESSE INTEGRADO"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / CONTROLES INTERNOS E CONFORMIDADE"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / ESTRUTURA DE GOVERNANÇA"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / ESTRUTURA ORGANIZACIONAL"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / GESTÃO DE DESIGN DE PROCESSOS DE NEGÓCIOS"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / INICIATIVAS ESTRATÉGICAS"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / MODELO DECISÓRIO"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / PLANEJAMENTO ESTRATÉGICO"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / PORTFÓLIO DE PRODUTOS E SERVIÇOS"
"[Processo corporativo] GOVERNANÇA E GESTÃO EMPRESARIAL / SUSTENTABILIDADE EMPRESARIAL"
"[Processo corporativo] GOVERNANÇA E SUPERVISÃO DE ENTIDADES LIGADAS / ESTRATÉGIA"
"[Processo corporativo] GOVERNANÇA E SUPERVISÃO DE ENTIDADES LIGADAS / MECANISMOS DE GOVERNANÇA"
"[Processo corporativo] GOVERNANÇA E SUPERVISÃO DE ENTIDADES LIGADAS / SUPERVISÃO"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / ESTRATÉGIAS DE INDUÇÃO"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / ESTRUTURAÇÃO E GOVERNANÇA DE DADOS PESSOAIS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE AMBIENTES DE SOLUÇÕES ANALÍTICAS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE CONTEÚDO E DOCUMENTOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE DADOS COMO ATIVO CORPORATIVO"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE DADOS PESSOAIS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE DOMÍNIOS E QUALIDADE DOS DADOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE FERRAMENTAS ANALÍTICAS E DADOS EXTERNOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DE MUDANÇAS DE MODELOS ANALÍTICOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DO CICLO DE VIDA DOS MODELOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DO PORTFÓLIO DE INICIATIVAS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GESTÃO DO PROCESSO DE MODELAGEM DE DADOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GOVERNANÇA DE DADOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / GOVERNANÇA DE MODELOS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / MONITORAÇÃO E RESPONSABILIZAÇÃO EM DADOS PESSOAIS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / PRIVACIDADE E PROTEÇÃO DE DADOS PESSOAIS"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / PROCESSO DE DESENVOLVIMENTO DE INTELIGÊNCIA ARTIFICIAL"
"[Processo corporativo] INTELIGÊNCIA ANALÍTICA / SEGURANÇA DE DADOS PESSOAIS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / BENS DE USO"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / BENS NÃO DE USO"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / COMPRAS, CONTRATAÇÕES E ADMINISTRAÇÃO DE CONTRATOS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / DISPONIBILIDADE, FUNCIONAMENTO E MANUTENÇÃO DAS INSTALAÇÕES FÍSICAS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / GESTÃO ARQUIVÍSTICA DE DOCUMENTOS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / GESTÃO DE FORNECEDORES"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / GESTÃO DE SISTEMAS DE PAGAMENTOS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / GESTÃO DE VALORES"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / GESTÃO DO PATRIMÔNIO"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / INFRAESTRUTURA"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / NUMERÁRIO"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / PROJETOS E AVALIAÇÕES DE ARQUITETURA E ENGENHARIA"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / SERVIÇOS DE COMPENSAÇÃO"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / SUPORTE OPERACIONAL"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / SUPORTE OPERACIONAL PARA PROCESSOS INTERNOS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / SUPORTE OPERACIONAL ÀS TRANSAÇÕES DE NEGÓCIOS"
"[Processo corporativo] LOGÍSTICA E SUPORTE OPERACIONAL / SUPRIMENTO DE BENS, MATERIAIS E SERVIÇOS"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / AÇÃO PROMOCIONAL"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / COMUNICAÇÃO DIGITAL"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / COMUNICAÇÃO INTERNA"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / ESTRATÉGIA DA MARCA"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / IMPRENSA"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / MERCADO INVESTIDOR"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / MONITORAMENTO E VALORAÇÃO DA MARCA"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / PATROCÍNIO"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / PLANEJAMENTO DA COMUNICAÇÃO"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / PUBLICIDADE E PROPAGANDA"
"[Processo corporativo] MARCA E COMUNICAÇÃO EMPRESARIAL / REDES SOCIAIS"
"[Processo corporativo] MEIOS DE PAGAMENTO / CONTAS"
"[Processo corporativo] MEIOS DE PAGAMENTO / INOVAÇÕES EM MEIOS DE PAGAMENTO"
"[Processo corporativo] MEIOS DE PAGAMENTO / NEGÓCIOS COM CARTÕES"
"[Processo corporativo] MEIOS DE PAGAMENTO / SOLUÇÕES EM PAGAMENTO E RECEBIMENTO"
"[Processo corporativo] MEIOS DE PAGAMENTO / TRANSFERÊNCIAS BANCÁRIAS"
"[Processo corporativo] MERCADO DE CAPITAIS / GESTÃO DAS CARTEIRAS TVM"
"[Processo corporativo] MERCADO DE CAPITAIS / NEGOCIAÇÃO DE TÍTULOS E VALORES MOBILIÁRIOS"
"[Processo corporativo] MERCADO DE CAPITAIS / OPERAÇÕES DE MERCADO DE CAPITAIS"
"[Processo corporativo] MERCADO DE CAPITAIS / SERVIÇOS FIDUCIÁRIOS"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / CRÉDITO PARA O SETOR PÚBLICO"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / DEPÓSITOS JUDICIAIS"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / GESTÃO DA REDE DE DISTRIBUIÇÃO ESPECIALIZADA E CANAIS DIGITAIS DE RELACIONAMENTO"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / GESTÃO DE CLIENTES SETOR PÚBLICO"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / GESTÃO DE FUNDOS E PROGRAMAS GOVERNAMENTAIS"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / PRODUTOS E SERVIÇOS PARA O SETOR PÚBLICO"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / RELACIONAMENTO COM O SETOR PÚBLICO"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / SOLUÇÕES PARA NEGÓCIOS SOCIAIS E POLÍTICAS PÚBLICAS"
"[Processo corporativo] NEGÓCIOS COM ENTES PÚBLICOS / SOLUÇÕES PARA O SETOR PÚBLICO"
"[Processo corporativo] NEGÓCIOS DE ATACADO / COBRANÇA E RECUPERAÇÃO DE CRÉDITO NO PILAR ATACADO"
"[Processo corporativo] NEGÓCIOS DE ATACADO / CRÉDITO ATACADO"
"[Processo corporativo] NEGÓCIOS DE ATACADO / GESTÃO DE CANAIS DO ATACADO"
"[Processo corporativo] NEGÓCIOS DE ATACADO / GESTÃO DE CLIENTES DO ATACADO"
"[Processo corporativo] NEGÓCIOS DE ATACADO / PRIVATE BANK"
"[Processo corporativo] NEGÓCIOS DE ATACADO / PRODUTOS, SERVIÇOS E SOLUÇÕES DE ATACADO"
"[Processo corporativo] NEGÓCIOS DE SEGURIDADE / ESTRATÉGIA DE NEGÓCIOS DE SEGURIDADE"
"[Processo corporativo] NEGÓCIOS DE SEGURIDADE / MONITORAMENTO DOS NEGÓCIOS DE SEGURIDADE"
"[Processo corporativo] NEGÓCIOS DE SEGURIDADE / OPERACIONALIZAÇÃO DE NEGÓCIOS DE SEGURIDADE"
"[Processo corporativo] NEGÓCIOS DIGITAIS / BANCO COMO SERVIÇO"
"[Processo corporativo] NEGÓCIOS DIGITAIS / ESTRATÉGIA E DESEMPENHO DE NEGÓCIOS DIGITAIS"
"[Processo corporativo] NEGÓCIOS DIGITAIS / OPEN FINANCE"
"[Processo corporativo] NEGÓCIOS DIGITAIS / PARCERIAS E INVESTIMENTOS EM STARTUPS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / ADQUIRIR, DESENVOLVER E IMPLEMENTAR SOLUÇÕES DE TI / ACQUIRE, DEVELOP AND IMPLEMENT IT SOLUTIONS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / ALINHAR, PLANEJAR E ORGANIZAR / ALIGN, PLAN AND ORGANIZE"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / CARD SERVICES"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / CLIENTES E CANAIS / CUSTOMERS AND CHANNELS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / CONTABILIDADE / ACCOUNTING"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / CONTROLES INTERNOS E COMPLIANCE / INTERNAL CONTROLS AND COMPLIANCE"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / CRÉDITO / CREDIT"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / DEPÓSITOS / DEPOSITS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / ENTREGA E APOIO / DELIVER AND SUPPORT"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / GESTÃO DE PESSOAS / HUMAN RESOURCES"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / GESTÃO DE PORTFÓLIO DE PRODUTOS E DE CONVÊNIOS INTERNACIONAIS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / GESTÃO DE REDE E DE ESTRATÉGIA DE ATUAÇÃO"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / GESTÃO DE RISCOS / RISK MANAGEMENT"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / GOVERNANÇA CORPORATIVA / CORPORATE GOVERNANCE"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / LOGÍSTICA / LOGISTICS"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / MONITORAR E AVALIAR / MONITOR AND EVALUATE"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / OPERACIONALIZAÇÃO DE CÂMBIO E COMÉRCIO EXTERIOR"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / OPERAÇÃO NO EXTERIOR"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / SEGURANÇA / CORPORATE SECURITY"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / SERVIÇOS / SERVICES"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / TECNOLOGIA DA INFORMAÇÃO E SEGURANÇA EM TI / INFORMATION TECHNOLOGY AND IT SECURITY"
"[Processo corporativo] NEGÓCIOS INTERNACIONAIS / TESOURARIA / TREASURY"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / GESTÃO DA ANTICORRUPÇÃO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / GESTÃO DA INTEGRIDADE"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / GESTÃO, ESTRATÉGIA E MODELAGEM"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / OPERACIONALIZAÇÃO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC  - PROCEDIMENTOS DE CONHEÇA SEU CLIENTE, FUNCIONÁRIO, PARCEIRO E PRESTADOR DE SERVIÇO TERCEIRIZADO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC - ACOMPANHAMENTO E CONTROLE"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC - ANÁLISE"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC - COMUNICAÇÃO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC - GOVERNANÇA"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PLD/FTC - MONITORAMENTO E SELEÇÃO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PREVENÇÃO À LAVAGEM DE DINHEIRO E AO FINANCIAMENTO DO TERRORISMO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PREVENÇÃO À OCULTAÇÃO DE ATIVOS FINANCEIROS"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PREVENÇÃO À PRÁTICA DE ATOS CONTRA A ADMINISTRAÇÃO PÚBLICA"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / PROCESSO ADMINISTRATIVO DE RESPONSABILIZAÇÃO"
"[Processo corporativo] PREVENÇÃO A ILÍCITOS FINANCEIROS E À CORRUPÇÃO / REPORTE"
"[Processo corporativo] SERVIÇOS JURÍDICOS / ACORDOS JUDICIAIS"
"[Processo corporativo] SERVIÇOS JURÍDICOS / ADVOCACIA CONSULTIVA"
"[Processo corporativo] SERVIÇOS JURÍDICOS / ADVOCACIA CONTENCIOSA"
"[Processo corporativo] SERVIÇOS JURÍDICOS / ADVOCACIA PREVENTIVA"
"[Processo corporativo] SERVIÇOS JURÍDICOS / DEPÓSITOS JUDICIAIS E RECURSAIS - BANCO PARTE E DEPOSITANTE"
"[Processo corporativo] SERVIÇOS JURÍDICOS / PROVISÃO PARA DEMANDAS CONTINGENTES (PDC)"
"[Processo corporativo] SERVIÇOS JURÍDICOS / SERVIÇOS ADMINISTRATIVO-JURÍDICOS"
"[Processo corporativo] SERVIÇOS JURÍDICOS / SERVIÇOS FINANCEIRO-JURÍDICOS"
"[Processo corporativo] SERVIÇOS JURÍDICOS / SERVIÇOS TÉCNICO-JURÍDICOS"
"[Processo corporativo] SERVIÇOS JURÍDICOS / TERCEIRIZAÇÃO DE SERVIÇOS JURÍDICOS"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / ACOMPANHAMENTO DO USO DA TECNOLOGIA DA INFORMAÇÃO"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / ALINHAMENTO, PLANEJAMENTO E ORGANIZAÇÃO DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / AQUISIÇÃO DE SOLUÇÕES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / AQUISIÇÃO, CONSTRUÇÃO E IMPLANTAÇÃO DE SOLUÇÕES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / DIRECIONAMENTO DO USO DA TECNOLOGIA DA INFORMAÇÃO"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / ENTREGA E SUPORTE DE SERVIÇOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GERENCIAMENTO DA RELAÇÃO ENTRE TI E NEGÓCIOS"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA ARQUITETURA DE DADOS"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA ARQUITETURA DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA CONTINUIDADE DOS SERVIÇOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA DISPONIBILIDADE DOS RECURSOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA ESTRATÉGIA DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA ESTRATÉGIA E ORÇAMENTO DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA INOVAÇÂO EM TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA QUALIDADE EM TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA SEGURANÇA EM SOLUÇÕES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DA SOLICITAÇÃO DE SERVIÇOS E INCIDENTES"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DAS INSTALAÇÕES FÍSICAS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE ATENDIMENTO A CLIENTES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE ATIVOS"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE CONFIGURAÇÃO"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE DEMANDAS E RELACIONAMENTOS COM CLIENTES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE DESEMPENHO E CAPACIDADE"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE FORNECEDORES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE MUDANÇAS E LIBERAÇÕES"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE PROBLEMAS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE PROGRAMAS E PROJETOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE RECURSOS HUMANOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DE RISCOS DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DO ARMAZENAMENTO"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DO FORNECIMENTO DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GESTÃO DO ORÇAMENTO DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / GOVERNANÇA E GESTÃO DA TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / IDENTIFICAÇÃO E DESENVOLVIMENTO DE SOLUÇÕES DE TI"
"[Processo corporativo] TECNOLOGIA DA INFORMAÇÃO / MONITORAR RISCOS, CONTROLES INTERNOS E CONFORMIDADE LEGAL DA TI"
"[Processo corporativo] TESOURARIA / CONTROLE E SUPORTE A OPERAÇÕES FINANCEIRAS"
"[Processo corporativo] TESOURARIA / GESTÃO DE CARTEIRAS E OPERAÇÕES FINANCEIRAS"
"[Processo corporativo] TESOURARIA / GESTÃO DO FLUXO DE CAIXA"
'''

In [10]:
resultado = f"{orientacao_prompt} # Formato Saída {formato_saida}, irregularidades: {irregularidades}, base de conhecimento {exemplo_rag_dinamica} "

In [11]:
resultado

'Você é um assistente de auditoria especializado em vincular processos corporativos de auditoria. Sua tarefa é classificar processos de apuração de irregularidades em categorias específicas de risco corporativo e retornar o resultado no formato JSON. As categorias de risco são:\n   1.Risco de Estratégia; 2.Risco de Crédito; 3.Risco Atuarial; 4.Risco de IRRBB; 5.Risco de Mercado; 6.Risco de Liquidez; 7.Risco de Contágio; 18.Risco Social, Ambiental e Climático; 45.Risco Cibernético; 48.Risco de Conformidade; 49.Risco de Terceiros; 50.Risco Legal; 776.Risco de Conduta; 777.Risco de Modelo; 778.Risco de Segurança; 779.Risco de TI\n   Classifique a apuração de irregularidades em uma ou mais das categorias de risco corporativo listadas acima. Para cada categoria, indique "Sim" ou "Não". Além disso, crie uma categoria chamada regra_classificacao que descreve qual a lógica que você usou para classificar.\n   Importante:\n   1) A base de conhecimento possui "Definição Principal" e "Definição Se

In [12]:
#estou limitando em 20 mil tokens seguraca, limit servico 128k

In [13]:
def limit_tokens(text, max_tokens=20000):
    if text is None:
        return '' 
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        # Re-encode apenas os tokens dentro do limite e decodifica de volta para texto
        text = encoding.decode(tokens[:max_tokens])
    return text


In [14]:
prompt = limit_tokens(resultado)

In [15]:
def assistenteManual(prompt):
    
    # fiz tudo manual p assimilar, servico disponível na plataforma tecnologia
    # aqui pode simular o retorno desejado e se aprofundar no processo e na informacao que procura resolver
    #pode simular o encadeamento de agentes entre outas opcoes
    
    
    url = 'http://acs-assist-sem-rag.nia.hm.bb.com.br/acs/llms/agent'
    resp = requests.post(url, data=json.dumps({
        "data": {
            "input": prompt,
            "intents": [{}],
            "entities": [{}],
            "context": {
                "config": {
                    "top_p": 0.95,
                    "max_tokens": 4000,
                    "temperature": 0.2
                }
            }
        }
    }), headers={'Content-type': 'application/json', 'accept': 'application/json'})

    # Convertendo a resposta em um dicionário
    resp_dict = json.loads(resp.text)

    # Extraindo o status
    status = resp_dict.get('status')
    print('status, retorno 200 ok, diferente contactar nia ', status)
    # simular retorno no swagger : http://acs-assist-sem-rag.nia.hm.bb.com.br/nia_semantica_acs/docs

    # Extraindo o resultado da resposta visualizar o que está acontecendo...
    texto_resposta = resp_dict["data"]["context"]["prompt_response"]["acs_llm_prompt_execution"]["result"]
    
    print('texto_resposta: ', texto_resposta)
    
    return texto_resposta

In [16]:
teste_caesp = assistenteManual(prompt)

status, retorno 200 ok, diferente contactar nia  200
texto_resposta:  {
    "regra_classificacao": "As irregularidades foram classificadas com base na natureza das atividades descritas, que envolvem principalmente questões de crédito, conformidade, conduta, segurança e riscos relacionados a tecnologia da informação.",
    "risco_estrategia": "Não",
    "risco_credito": "Sim",
    "risco_atuarial": "Não",
    "risco_IRRBB": "Não",
    "risco_mercado": "Não",
    "risco_liquidez": "Não",
    "risco_contagio": "Não",
    "risco_social_ambiental_climatico": "Não",
    "risco_cibernetico": "Sim",
    "risco_conformidade": "Sim",
    "risco_terceiros": "Sim",
    "risco_legal": "Sim",
    "risco_conduta": "Sim",
    "risco_modelo": "Não",
    "risco_seguranca": "Sim",
    "risco_ti": "Sim"
}


In [17]:
display(JSON(teste_caesp))

/usr/local/lib/python3.8/dist-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>